In [1]:
import sys
import os
import numpy as np
from datetime import datetime
import xarray as xr
import iris
import cartopy.crs as ccrs
from iris.analysis.cartography import unrotate_pole, rotate_pole
import pandas as pd

# Test code for file processing 
Note - I've converted this into a python script (um2nc_RNS.py) to run in command line: 
See: /home/563/slf563/simple_iris_wrap_RNS.sh

In [2]:
date = '20240113T0000Z'
fdir = '/scratch/jk72/slf563/cylc-run/u-di850/share/cycle/{}/Regn1/resn_1/RAL3P2_glomap/um/'.format(date)
hour = '030'
clat = -64.8
clon = 141
target_lat = -64.8 #  -68.5762 # For Davis 
target_lon = 141 # 77.9696

In [29]:
fname = 'umnsaa_psurf{}'.format(hour)
cubes = iris.load(fdir+fname)

In [30]:
cubes

[<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of air_temperature / (K) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of dew_point_temperature / (K) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of relative_humidity / (%) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of sea_ice_area_fraction / (1) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of specific_humidity / (1) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of surface_air_pressure / (Pa) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of surface_downwelling_longwave_flux_in_air / (W m-2) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of surface_downwelling_shortwave_flux_in_air / (W m-2) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of surface_downwelling_shortwave_flux_in_air_assuming_clear_sky / (W m-2) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of surface_temperature / (K) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of wind_speed / (m s-1) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of x_wind / (m s-1) (time: 36; grid_latitude: 450; grid_longitude: 400)>,
<iris 'Cube' of y_wind / (m s-1) (time: 36; grid_latitude: 451; grid_longitude: 400)>]

In [31]:
def filter_exact_lat_lon(data,target_lat,target_lon,central_lat,central_lon): 
    grid_lon, grid_lat = rotate_pole(np.array(target_lon), np.array(target_lat), central_lon, central_lat+90)
    grid_lon = grid_lon % 360
    point_data = data
    if 'grid_latitude' in data.dims: 
        point_data = point_data.sel(grid_latitude = grid_lat, grid_longitude = grid_lon, method='nearest')
    if 'grid_latitude1' in data.dims: 
        point_data = point_data.sel(grid_latitude1 = grid_lat, grid_longitude1 = grid_lon, method='nearest')
    if 'grid_latitude2' in data.dims: 
        point_data = point_data.sel(grid_latitude2 = grid_lat, grid_longitude2 = grid_lon, method='nearest')

    return point_data


In [32]:
dataout = xr.Dataset()
for i,cube in enumerate(cubes[:]): 

    rotated_lons, rotated_lats = cube.coord('grid_longitude').points, cube.coord('grid_latitude').points
    x, y = np.meshgrid(rotated_lons, rotated_lats)
    lons, lats = unrotate_pole(x, y, clon, clat+90)
    
    data = xr.DataArray.from_iris(cube)
    data = data.assign_coords({'latitude':(('grid_latitude','grid_longitude'),lats)})
    data = data.assign_coords({'longitude':(('grid_latitude','grid_longitude'),lons)})
    
    if i > 0 and data.grid_latitude[0].values != dataout.grid_latitude[0].values:
        data = data.rename({'grid_latitude':'grid_latitude1'})
        data = data.rename({'grid_longitude':'grid_longitude1'})
        data = data.rename({'latitude':'latitude1'})
        data = data.rename({'longitude':'longitude1'})

    if 'grid_longitude' in data.dims and i > 0 and data.grid_longitude[0].values != dataout.grid_longitude[0].values:
        data = data.rename({'grid_longitude':'grid_longitude2'})
        data = data.rename({'grid_latitude':'grid_latitude2'})
        data = data.rename({'latitude':'latitude2'})
        data = data.rename({'longitude':'longitude2'})

    if 'time' in list(data.coords) and 'time' in list(dataout.coords):    
        if data.time[0].values != dataout.time[0].values:
            data = data.rename({'time':'time1'})
            data = data.rename({'forecast_period':'forecast_period1'})
            
    
    if 'level_height' in list(data.coords) and 'level_height' in list(dataout.coords):    
        if data.level_height[0].values != dataout.level_height[0].values:
            data = data.rename({'level_height':'level_height1'})
            data = data.rename({'model_level_number':'model_level_number1'})

    if 'sigma' in list(data.coords) and 'sigma' in list(dataout.coords):
        if data.sigma[0].values != dataout.sigma[0].values:
            data = data.rename({'sigma':'sigma1'})

    if (data.attrs['STASH'].item == 229) and (data.attrs['STASH'].section == 15): 
        data = data.rename('potential_vorticity')
    if data.attrs['STASH'].item == 293 and data.attrs['STASH'].section == 30:     
        data = data.rename('w_component_of_wind')
    if data.attrs['STASH'].item == 294 and data.attrs['STASH'].section == 30: 
        data = data.rename('temperature')
    if data.attrs['STASH'].item == 295 and data.attrs['STASH'].section == 30:     
        data = data.rename('specific_humidity')
    if data.attrs['STASH'].item == 296 and data.attrs['STASH'].section == 30: 
        data = data.rename('relative_humidity')
    if data.attrs['STASH'].item == 297 and data.attrs['STASH'].section == 30: 
        data = data.rename('geopotential_height')
    if data.attrs['STASH'].item == 304 and data.attrs['STASH'].section == 30: 
        data = data.rename('heavyside_function')  

    if data.attrs['STASH'].item == 401 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_soluble_nucleation_mode_aerosol')  
    if data.attrs['STASH'].item == 402 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_soluble_aitken_mode_aerosol')  
    if data.attrs['STASH'].item == 403 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_soluble_accumulation_mode_aerosol')  
    if data.attrs['STASH'].item == 404 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_soluble_course_mode_aerosol')  
    if data.attrs['STASH'].item == 405 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_insoluble_aitken_mode_aerosol')  
    if data.attrs['STASH'].item == 406 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_insoluble_aitken_mode_aerosol')  
    if data.attrs['STASH'].item == 407 and data.attrs['STASH'].section == 38: 
        data = data.rename('dry_particle_diameter_insoluble_aitken_mode_aerosol')  

    if data.attrs['STASH'].item == 96 and data.attrs['STASH'].section == 0: 
        data = data.rename('ocean_near_surface_chlorophyll_km_per_m3')  
    if data.attrs['STASH'].item == 437 and data.attrs['STASH'].section == 38: 
        data = data.rename('condensation_nuclei_number_concentration_greater_than_3nm_dry_diameter_per_cm3')  
    if data.attrs['STASH'].item == 439 and data.attrs['STASH'].section == 38: 
        data = data.rename('CCN_greater_than_50nm_dry_diameter_AIT+ACC+COA_per_cm3')  

    if data.attrs['STASH'].item == 230 and data.attrs['STASH'].section == 3: 
        data = data.rename({'height':'height1'})  
    if data.attrs['STASH'].item == 209 and data.attrs['STASH'].section == 3: 
        data = data.rename({'height':'height1'}) 
    if data.attrs['STASH'].item == 210 and data.attrs['STASH'].section == 3: 
        data = data.rename({'height':'height1'}) 

    comp = dict(zlib=True, complevel=5)
    data.encoding.update(comp) 

    dataout = dataout.merge(data)
    
dataout = dataout.assign_attrs({'central lat of rotated grid':clat,
                                'central lon of rotated grid':clon,
                                'history':'Data generated and processed by S. Fiddes sonya.fiddes@utas.edu.au {}'.format(datetime.today().date())})

point_data = filter_exact_lat_lon(dataout,target_lat,target_lon,clat,clon)

In [33]:
dataout

<xarray.Dataset>
Dimensions:                                                       (time: 36,
                                                                   grid_latitude: 450,
                                                                   grid_longitude: 400,
                                                                   grid_latitude2: 450,
                                                                   grid_longitude2: 400,
                                                                   grid_latitude1: 451,
                                                                   grid_longitude1: 400)
Coordinates: (12/17)
  * time                                                          (time) datetime64[ns] ...
  * grid_latitude                                                 (grid_latitude) float32 ...
  * grid_longitude                                                (grid_longitude) float32 ...
    forecast_reference_time                                       datetime64[ns] ...
    forecast_period                                               (time) timedelta64[ns] ...
    latitude                                                      (grid_latitude, grid_longitude) float64 ...
    ...                                                            ...
    latitude2                                                     (grid_latitude2, grid_longitude2) float64 ...
    longitude2                                                    (grid_latitude2, grid_longitude2) float64 ...
  * grid_latitude1                                                (grid_latitude1) float32 ...
  * grid_longitude1                                               (grid_longitude1) float32 ...
    latitude1                                                     (grid_latitude1, grid_longitude1) float64 ...
    longitude1                                                    (grid_latitude1, grid_longitude1) float64 ...
Data variables: (12/14)
    air_pressure_at_sea_level                                     (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    air_temperature                                               (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    dew_point_temperature                                         (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    relative_humidity                                             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    sea_ice_area_fraction                                         (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    specific_humidity                                             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    ...                                                            ...
    surface_downwelling_shortwave_flux_in_air                     (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air_assuming_clear_sky  (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    surface_temperature                                           (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    wind_speed                                                    (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    x_wind                                                        (time, grid_latitude2, grid_longitude2) float32 dask.array<chunksize=(1, 450, 400), meta=np.ndarray>
    y_wind                                                        (time, grid_latitude1, grid_longitude1) float32 dask.array<chunksize=(1, 451, 400), meta=np.ndarray

# Test code for extracting along ship track

In [34]:
extract_ship='MISO'

In [54]:
if extract_ship!='None': 
    print('extracting data along '+extract_ship+' shiptrack') 
    if extract_ship == 'MISO':
        # This is the 1 min underway data - hopefully all RVI underway data is similar in structure
        shiptrack = pd.read_csv('/g/data/jk72/slf563/OBS/campaigns/MISO/IN2024_V01_uwy_data.csv',index_col=0)
        shiptrack.index = pd.DatetimeIndex(shiptrack.index)
        shiptrack = shiptrack.drop(shiptrack[shiptrack['Latitude (degree_north)']>-62.505].index)
        shiptrack = shiptrack.drop(shiptrack[shiptrack['Longitude (degree_east)']<137.3].index)
        shiptrack = shiptrack.drop(shiptrack[shiptrack['Longitude (degree_east)']>145.4].index)
        
    #elif extract_ship == 'new ship track': 
        
    else: 
        'Ship track not recognised - add code above for desired ship track'
    
    time = pd.DatetimeIndex(dataout.time) # convert model time to datetimeindex to loop through and 

    ship_data = xr.Dataset()
    for t in time[-2:]: 
        if t > shiptrack.index[0] and t < shiptrack.index[-1]:
            print('voyage data found for this time range')

            lat = shiptrack.loc['{}-{:02}-{} {:02}{:02}'.format(
                t.year,t.month,t.day,t.hour,t.minute)]['Latitude (degree_north)']  
            lon = shiptrack.loc['{}-{:02}-{} {:02}{:02}'.format(
                t.year,t.month,t.day,t.hour,t.minute)]['Longitude (degree_east)']
            
            grid_lon, grid_lat = rotate_pole(np.array(lon), np.array(lat), clon, clat+90)
            grid_lon = grid_lon % 360
            
            point = dataout.sel(time=t)
            if 'time1' in dataout.dims:
                point = point.sel(time1=t,method='nearest')
                point = point.drop_vars('time1')
                point = point.drop_vars('forecast_period1')
            if 'grid_latitude' in dataout.dims:
                point = point.sel(grid_latitude = grid_lat, grid_longitude = grid_lon, method='nearest')
            if 'grid_latitude1' in dataout.dims:
                point = point.sel(grid_latitude1 = grid_lat, grid_longitude1 = grid_lon, method='nearest')
            if 'grid_latitude2' in dataout.dims:
                point = point.sel(grid_latitude2 = grid_lat, grid_longitude2 = grid_lon, method='nearest')

            reformatted_point = xr.Dataset()
            for key in list(point.keys()):
                tmp = point[key]
                if 'grid_latitude' in point[key].dims: 
                    tmp = tmp.isel(grid_latitude=0,grid_longitude=0)
                if 'grid_latitude1' in point[key].dims: 
                    tmp = tmp.isel(grid_latitude1=0,grid_longitude1=0)
                if 'grid_latitude2' in point[key].dims: 
                    tmp = tmp.isel(grid_latitude2=0,grid_longitude2=0)
                        
                tmp = tmp.expand_dims('time')
                reformatted_point[key] = tmp
                
            point = reformatted_point
    
            # Drop the current grid lat/lons because they are a bit useless and convlute things
            if 'grid_latitude' in list(point.coords):
                point = point.drop_vars(['grid_latitude','grid_longitude'])
            if 'grid_latitude1' in list(point.coords):
                point = point.drop_vars(['grid_latitude1','grid_longitude1'])
            if 'grid_latitude2' in list(point.coords):
                point = point.drop_vars(['grid_latitude2','grid_longitude2'])
    
            # but then we need to include the grid lat/lon as a dimension for the aerosol conversions to work, so 
            # add in as a dummy dim, with only 1 point which equals 0 for all.. 
            point = point.expand_dims('grid_latitude')
            point = point.expand_dims('grid_longitude')

            # rearrange dims to be time, level, lat, lon
            if ('model_level_number' in point.dims) and ('model_level_number1' in point.dims):
                point = point.transpose('time','model_level_number','model_level_number1','grid_latitude','grid_longitude')
            elif 'model_level_number' in point.dims: 
                point = point.transpose('time','model_level_number','grid_latitude','grid_longitude')
            else: 
                point = point.transpose('time','grid_latitude','grid_longitude')
            
            # Add ship location as coordinate...
            point = point.assign_coords({'ship_latitude':lat})
            point['ship_latitude'] = point['ship_latitude'].ship_latitude.expand_dims('time')
            point = point.assign_coords({'ship_longitude':lon})
            point['ship_longitude'] = point['ship_longitude'].ship_longitude.expand_dims('time')
            point = point.drop_vars('forecast_period')
            
            ship_data = ship_data.merge(point)

    if len(ship_data)>0:
        print('write out data here')
    #ship_data.to_netcdf('{}_{}{}'.format(fout[:-3],extract_ship,fout[-3:]))

extracting data along MISO shiptrack
voyage data found for this time range
voyage data found for this time range
write out data here


In [56]:
ship_data

<xarray.Dataset>
Dimensions:                                                       (time: 2,
                                                                   grid_latitude: 1,
                                                                   grid_longitude: 1)
Coordinates:
  * time                                                          (time) datetime64[ns] ...
    forecast_reference_time                                       datetime64[ns] ...
    latitude                                                      float64 -66.48
    longitude                                                     float64 145.0
    height                                                        float64 1.5
    height1                                                       float64 10.0
    latitude2                                                     float64 -66.48
    longitude2                                                    float64 144.9
    latitude1                                                     float64 -66.47
    longitude1                                                    float64 145.0
    ship_latitude                                                 (time) float64 ...
    ship_longitude                                                (time) float64 ...
Dimensions without coordinates: grid_latitude, grid_longitude
Data variables: (12/14)
    air_pressure_at_sea_level                                     (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    air_temperature                                               (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    dew_point_temperature                                         (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    relative_humidity                                             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    sea_ice_area_fraction                                         (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    specific_humidity                                             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    ...                                                            ...
    surface_downwelling_shortwave_flux_in_air                     (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air_assuming_clear_sky  (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    surface_temperature                                           (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    wind_speed                                                    (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    x_wind                                                        (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>
    y_wind                                                        (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 1, 1), meta=np.ndarray>

### This code can pull out just 1 degree around a point - but I havent implemented it anywhere yet. 

In [25]:
grid_lon, grid_lat = rotate_pole(np.array(target_lon-1), np.array(target_lat+1), 141, clat+90)
point_data = dataout
grid_lon = grid_lon % 360
if 'grid_latitude' in dataout.dims: 
    point_data = point_data.sel(grid_latitude = slice(grid_lat[0]-0.5,grid_lat[0]+0.5), 
                   grid_longitude = slice(grid_lon[0]-0.5,grid_lon[0]+0.5))
if 'grid_latitude1' in dataout.dims: 
    point_data = point_data.sel(grid_latitude1 = slice(grid_lat[0]-0.5,grid_lat[0]+0.5), 
                   grid_longitude1 = slice(grid_lon[0]-0.5,grid_lon[0]+0.5))
if 'grid_latitude2' in dataout.dims: 
    point_data = point_data.sel(grid_latitude2 = slice(grid_lat[0]-0.5,grid_lat[0]+0.5), 
                   grid_longitude2 = slice(grid_lon[0]-0.5,grid_lon[0]+0.5))
